In [4]:
import pandas as pd
import numpy as np

In [5]:
df_bat = pd.read_csv(r'C:\Local Disk F\ML\Campus SuperBowl Challenge 2024\AMEX_2024\batsman_level_scorecard.csv')

In [6]:
df_bat.head()

,match id,batsman,batsman_id,batsman_details,is_batsman_captain,is_batsman_keeper,inning,runs,balls_faced,over_faced_first,...,out_by_bowler,out_by_fielder,bowler_id,bowler_details,is_bowler_keeper,is_bowler_captain,strike_rate,Fours,Sixes,match_dt
0,8638034,KD Ce,7907451.0,NZ:Right-hand bat:Right-arm medium-fast:,0.0,0.0,1,7,5,1.1,...,JS Nm,TM Jn,2486896.0,NZ:Left-hand bat:Right-arm medium-fast:,0.0,0.0,140.00,1.0,NaN,2021-01-01
1,8638034,TL St,4381761.0,NZ:Right-hand bat:None:,0.0,1.0,1,46,46,1.2,...,R Ra,MG Bl,6718382.0,NZ:Left-hand bat:Slow left-arm orthodox:,0.0,0.0,100.00,4.0,1.0,2021-01-01
2,8638034,HR Cr,4949790.0,NZ:Right-hand bat:Right-arm offbreak:,0.0,0.0,1,9,10,12.1,...,PF Yd,R Ra,4950294.0,NZ:Right-hand bat:Legbreak googly:,0.0,0.0,90.00,NaN,NaN,2021-01-01
3,8638034,BR Hn,3834305.0,NZ:Right-hand bat:Right-arm medium-fast:,0.0,0.0,1,28,22,13.6,...,HK Bt,FH An,1585464.0,NZ:Left-hand bat:Right-arm medium-fast:,0.0,0.0,127.27,3.0,NaN,2021-01-01
4,8638034,SC Kn,3776849.0,NZ:Right-hand bat:Right-arm fast-medium:,0.0,0.0,1,18,13,17.2,...,NaN,NaN,NaN,NaN,NaN,NaN,138.46,NaN,1.0,2021-01-01


In [3]:
df_bat.columns

Index(['match id', 'batsman', 'batsman_id', 'batsman_details',
       'is_batsman_captain', 'is_batsman_keeper', 'inning', 'runs',
       'balls_faced', 'over_faced_first', 'wicket kind', 'out_by_bowler',
       'out_by_fielder', 'bowler_id', 'bowler_details', 'is_bowler_keeper',
       'is_bowler_captain', 'strike_rate', 'Fours', 'Sixes', 'match_dt'],
      dtype='object')

In [4]:
df_bat[['batsman_details', 'bowler_details']].head()

,batsman_details,bowler_details
0,NZ:Right-hand bat:Right-arm medium-fast:,NZ:Left-hand bat:Right-arm medium-fast:
1,NZ:Right-hand bat:None:,NZ:Left-hand bat:Slow left-arm orthodox:
2,NZ:Right-hand bat:Right-arm offbreak:,NZ:Right-hand bat:Legbreak googly:
3,NZ:Right-hand bat:Right-arm medium-fast:,NZ:Left-hand bat:Right-arm medium-fast:
4,NZ:Right-hand bat:Right-arm fast-medium:,NaN


In [5]:
df_bat[['batsman_country', 'batsman_bat', 'batsman_bowl', 'extra']] = df_bat['batsman_details'].str.split(':', expand=True)

# Split the 'bowler_details' column into separate columns
df_bat[['bowler_country', 'bowler_bat', 'bowler_bowl', 'extra2']] = df_bat['bowler_details'].str.split(':', expand=True)

# Drop the original columns
df_bat.drop(columns=['batsman_details', 'bowler_details', 'extra', 'extra2'], inplace=True)

In [6]:
from sklearn.preprocessing import MinMaxScaler

In [7]:
# Assuming your dataset is in a DataFrame named df

# Aggregate the data
batsman_stats = df_bat.groupby('batsman_id').agg({
    'runs': 'sum',
    'balls_faced': 'sum',
    'Fours': 'sum',
    'Sixes': 'sum',
    'match id': 'count'  # Assuming each row represents one inning
}).reset_index()

# Rename columns for clarity
batsman_stats.rename(columns={'match id': 'inning'}, inplace=True)

scaler = MinMaxScaler(feature_range=(0, 10))

# Calculate additional metrics
batsman_stats['average_runs'] = batsman_stats['runs'] / batsman_stats['inning']
batsman_stats['average_runs'] = scaler.fit_transform(batsman_stats[['average_runs']])

batsman_stats['strike_rate'] = (batsman_stats['runs'] / batsman_stats['balls_faced'])
batsman_stats['strike_rate'] = scaler.fit_transform(batsman_stats[['strike_rate']])

batsman_stats['average_fours'] = batsman_stats['Fours'] / batsman_stats['inning']
batsman_stats['average_fours'] = scaler.fit_transform(batsman_stats[['average_fours']])

batsman_stats['average_sixes'] = batsman_stats['Sixes'] / batsman_stats['inning']
batsman_stats['average_sixes'] = scaler.fit_transform(batsman_stats[['average_sixes']])

# Define the performance score (example weights)
batsman_stats['bat_performance_score'] = (
    batsman_stats['average_runs'] * 0.5 +
    batsman_stats['strike_rate'] * 0.3 +
    batsman_stats['average_fours'] * 0.1 +
    batsman_stats['average_sixes'] * 0.1
)

# Sort by performance score to see the top performers
batsman_stats = batsman_stats.sort_values(by='bat_performance_score', ascending=False)

# Display the result
print(batsman_stats)


      batsman_id  runs  balls_faced  Fours  Sixes  inning  average_runs  \
1833   8392194.0   111           71   14.0    3.0       1     10.000000   
1724   8142084.0    74           58   11.0    0.0       1      6.666667   
481    2789646.0   283          153   32.0   12.0       5      5.099099   
857    4170543.0   163          110   16.0    7.0       3      4.894895   
1132   5577676.0   122           98   11.0    3.0       2      5.495495   
...          ...   ...          ...    ...    ...     ...           ...   
1985   8931572.0     0            2    0.0    0.0       1      0.000000   
1345   6931140.0     0            1    0.0    0.0       1      0.000000   
297    2154732.0     0            5    0.0    0.0       2      0.000000   
1349   7005550.0     0            1    0.0    0.0       1      0.000000   
1569   7881866.0     0            1    0.0    0.0       1      0.000000   

      strike_rate  average_fours  average_sixes  bat_performance_score  
1833     2.605634      10.

In [8]:
batsman_stats['bat_performance_score'] = scaler.fit_transform(batsman_stats[['bat_performance_score']])

In [9]:
batsman_stats.head()

,batsman_id,runs,balls_faced,Fours,Sixes,inning,average_runs,strike_rate,average_fours,average_sixes,bat_performance_score
1833,8392194.0,111,71,14.0,3.0,1,10.000000,2.605634,10.000000,7.500000,10.000000
1724,8142084.0,74,58,11.0,0.0,1,6.666667,2.126437,7.857143,0.000000,6.315951
481,2789646.0,283,153,32.0,12.0,5,5.099099,3.082789,4.571429,6.000000,6.016616
857,4170543.0,163,110,16.0,7.0,3,4.894895,2.469697,3.809524,5.833333,5.513560
1132,5577676.0,122,98,11.0,3.0,2,5.495495,2.074830,3.928571,3.750000,5.494190


In [10]:
batsman_stats.to_csv(r'C:\Local Disk F\ML\Campus SuperBowl Challenge 2024\Data\batsman_stats.csv', index = False)